# Datasets Creation: OSM restaurants in Milano

This tutorial explain step-by-step the R script used to generate the dataset for the GWAP-enabler tutorial. The dataset consists of all the OpenStreetMap restaurants in the Milano area (Italy). 

By running this script you can generate the INSERT queries to populate tables **resource**, **topic** and **resource_has_topic**.
The script consists in the following 3 steps:
1. extracting data from OpenStreetMap
2. cleaning and filtering data
3. SQL query generation

This script can be also customized to extract different kinds of POIs from different bounding boxes.

## STEP 1: extracting data from OSM

First of all you need to import a set of functions to query OpenStreetMap and to retrieve data. These functions are defined in the file `osm-utilities-tutorial.r`. Load them by running the following commands:

In [ ]:
source("osm-utilities-tutorial.r")
loadRPackage()
loadRequiredPackages()

Then you have to define the geographical area of your analysis, by setting the coordinates of the bounding box from which you want to extract data. 
The coordinates of the bounding box can be extracted from the OpenStreetMap website https://www.openstreetmap.org/export#map=16/45.5210/9.2184, which can convert the drawing of a bounding box into a set of reference coordinates. The coordinates specified in the code above covers the Milano area, but you are free to modify them, accordingly to your needs. The coordinates must be speecified in the following order: South, West, North, East.

In [20]:
swne<-c(45.4785, 9.1442, 45.5397, 9.2792)

The other parameter you can specify is the set of restaurants' attributes you want to retrieve. You can modify the code below by listing all the attributes you are interested in. 
You can find the list of all the available attributes for a given type of resource by querying https://overpass-turbo.eu/ and analysing the data result.

In [22]:
col<-"::id,::lat,::lon,name,cuisine,website"

At this point you are ready to create and execute the query. The data retrieved are saved in a .csv file named `output-file.csv` and stored in memory in the variable `all.data`.

In [23]:
query<-getNodeRestaurantOverpassQueryCSV(swne= swne, columns = col)
getOsmData(query.string = query, output.file = "output-file.csv",slowdown = F)
all.data<-read.csv("output-file.csv", sep = "\t")

_OPTIONAL_: If you want to modify the type of OSM POIs retrieved you have to open the file "osm-utilities-tutorial.r" with a text editor and modify the function `getNodeRestaurantOverpassQueryCSV`. In particular you have to change the type of OSM node retrieved by changing `node[amenity=restaurant]` with another key-value pair using the following syntax `node[key=value]`. The complete list of OSM tag can be found here: https://wiki.openstreetmap.org/wiki/Map_Features.

In [24]:
getNodeRestaurantOverpassQueryCSV <- function(coords=NULL, swne=c(), columns=c()){
  if(is.null(coords)){
    bbox.string = paste(swne, collapse=",")
    query <- paste0("[out:csv(", columns, ")];node[amenity=restaurant](", bbox.string, "); out;")
  }else{
    coords = t(coords[,2:1])
    bounds.string = paste(coords, collapse = " ")
    query <- paste0("[out:csv(", columns, ")];node[amenity=restaurant](poly:\"", bounds.string, "\"); out;")
  }
  return(query)
}

## STEP 2: Cleaning and filtering data

Once data has been retrieved, a filtering and cleaning process is required to generate the desired dataset.
The aim is to collect all the Milano restaurants with a valid name and to split data into a set of restaurants for which the type of cuisine is already known (_Ground Truth set_) and a set of restaurants for which the type of cuisine has to be discovered.  

First of all, only the restaurants with a valid name are kept:

In [25]:
filtered<-all.data[!all.data$name =="",]

By analysing the target attribute (the `cuisine` column), it is evident that some restaurants have multiple tags, separated by ";" or "," or ":". Since the GWAP-enabler can't mangae multiple classification, the simplest solution can be to keep only the first tag and delete all the others. More sophisticated solutions can be adopted and are left as exercise. 

In [26]:
filtered$cuisine<-as.character(filtered$cuisine)
filtered$cuisine<-gsub("[;,:]\\s*[A-z]*","",filtered$cuisine)

Next step is the definition of the output categories (types of `cuisine`) with the corresponding frequency of occurence in the dataset (prior probability). 
Here you have two options:
* select the overall top n categories (overall ranking taken from https://taginfo.openstreetmap.org/keys/cuisine#values with the corresponding frequencies)

In [27]:
label<- c("regional", "pizza", "burger", "italian", "chinese", "sandwich", "mexican", "japanese", "indian", "kebab")
prior<- c(0.114, 0.0972, 0.0824, 0.0671, 0.0493, 0.0344, 0.0268, 0.0239, 0.0213, 0.0211) 
categories<-data.frame(label, prior)

* select the top n categories from the specific use case (only the restaurants in the area considered)

In [28]:
table.categories<-as.data.frame(table(filtered$cuisine))
table.categories<-table.categories[!table.categories$Var1=="",]
table.categories<-table.categories[with(table.categories, order(-Freq)), ]
table.categories$prior<-round(table.categories$Freq/sum(table.categories$Freq),3)
categories<-table.categories[c(1:9), c("Var1","prior")]
names(categories)<-c("label", "prior")

Using the set of output categories defined, you can split the dataset in two sets: 
* _Ground Truth restaurant (`gt`)_: restaurants already labelled with one of the selected categories     
* _Restaurant to be classified (`to.classify`)_: restaurants with no value for the 'cuisine' attribute

All the restaurants are also store together in the `resource` variable to simplify the subsequent operations.

In [29]:
gt<-filtered[filtered$cuisine %in% categories$label, ]
to.classify<-filtered[! filtered$X.id %in% gt$X.id & filtered$cuisine =="", ]
resource<-gt
resource<-rbind(resource, to.classify)

## STEP 3: SQL query generation

The following part of the script generates the SQL INSERT query to populate the database. The resulting queries are exported in .sql files.

* INSERT query to populate the table <b>resource</b>. All the queries are saved in file `2_Insert_Resource.sql`

In [30]:
query.resource<-character()
query.resource<-c("LOCK TABLES resource WRITE;")
query.resource<-c(query.resource, "INSERT INTO resource (refId, lat, `long`, orderBy, label, url) VALUES")
    
for(i in 1:nrow(resource)){
  if(i != nrow(resource)){
    q<-paste0("('http://www.openstreetmap.org/node/", resource[i,]$X.id, "'," , resource[i,]$X.lat, "," , resource[i,]$X.lon, ", rand(),'", gsub("'","''",resource[i,]$name), "', ''),")
    query.resource<-c(query.resource, q)
  }else{
    q<-paste0("('http://www.openstreetmap.org/node/", resource[i,]$X.id, "'," , resource[i,]$X.lat, "," , resource[i,]$X.lon, ", rand(),'", gsub("'","''",resource[i,]$name), "', '');")
    query.resource<-c(query.resource, q)
  }
}    
    
query.resource<-c(query.resource, "UNLOCK TABLES;")
write(query.resource, file="2_Insert_Resource.sql")  # all the resource INSERT queries

* INSERT query to populate the table <b>topic</b>. All the queries are saved in file `3_Insert_Topic.sql`

In [ ]:
query.topic<-character()
query.topic<-c("LOCK TABLES topic WRITE;")
query.topic<-c(query.topic, "INSERT INTO topic (refId, value, label, weight, url) VALUES")
    
for(ct in 1:nrow(categories)){
   if(ct != nrow(categories)){
      qt<-paste0("('", categories$label[ct], "','" , categories$label[ct], "','" , categories$label[ct], "',", categories$prior[ct], ", ''),")
      query.topic<-c(query.topic, qt)
   }else{
      qt<-paste0("('", categories$label[ct], "','" , categories$label[ct], "','" , categories$label[ct], "',", categories$prior[ct], ", '');")
      query.topic<-c(query.topic, qt)
  }
 }
query.topic<-c(query.topic, "UNLOCK TABLES;")
write(query.topic, file="3_Insert_Topic.sql") # all the topic INSERT queries

* INSERT query to populate the table <b>resource_has_topic</b>. All the queries are saved in file `4_Insert_Resource_has_Topic.sql`. GT resources have score 2 for the assigned topic, whereas the resources to be classified are initialized with score 0 for all the possible topics.

In [32]:
#c.1) GT resources: only one row of the correspondig topic with score 2
query.rht.gt<-character()
query.rht.gt<-c("LOCK TABLES resource_has_topic WRITE, resource READ, topic READ;")
      
for(t in 1:nrow(gt)){
  qgt<-paste0("INSERT INTO resource_has_topic (idResource, idTopic, score) VALUES ((SELECT idResource FROM resource WHERE refId = 'http://www.openstreetmap.org/node/", gt$X.id[t], "'), (SELECT idTopic FROM topic WHERE refId ='", gt$cuisine[t], "'), 2);")
  query.rht.gt<-c(query.rht.gt,qgt)
}
     
#c.2) toClassify resources: one row for each topic with score 0
query.rht.toclassify<-character()

for(w in 1:nrow(to.classify)){
  for(f in 1:nrow(categories)){
     qtc<-paste0("INSERT INTO resource_has_topic (idResource, idTopic, score) VALUES ((SELECT idResource FROM resource WHERE refId = 'http://www.openstreetmap.org/node/", to.classify$X.id[w], "'), (SELECT idTopic FROM topic WHERE refId ='", categories$label[f], "'), 0);")
     query.rht.toclassify<-c(query.rht.toclassify,qtc)
  }
}
query.rht.toclassify<-c(query.rht.toclassify, "UNLOCK TABLES;")
      
#save both the GT and toClassify resource_has_topic in a single file
query.rht<-c(query.rht.gt, query.rht.toclassify)
write(query.rht, file="4_Insert_Resource_has_Topic.sql")  # all the resource_has_topic INSERT queries 

Now the datasets to populate your database are ready! 